In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
from getdist import plots, MCSamples
import scipy.integrate as integrate
# import planckStyle
import pylab
import numpy as np

params1 = {'backend': 'pdf',
               'axes.labelsize': 20,
               'xtick.labelsize': 20,
               'ytick.labelsize': 20,
               'legend.fontsize': 16,
               'lines.markersize': 6,
               'font.size': 25,
               'text.usetex': True}#
pylab.rcParams.update(params1)


In [2]:
dir_name= '/home/isidro/Downloads/chains/'

In [4]:
roots = ['Restrained_phy_CBAO+HD+Pantheon+Planck_15_nested_multi']

g= plots.getSinglePlotter(chain_dir = dir_name, width_inch=6, 
                          analysis_settings={'smooth_scale_1D': 0.4,'smooth_scale_2D': 0.5, 'ignore_rows': 0.2})

# g.settings.axis_tick_rotation = 75
# g.settings.axes_fontsize    = 20
# g.settings.lab_fontsize     = 20
# g.settings.alpha_filled_add = 0.9

In [5]:
samples = g.sampleAnalyser.samplesForRoot('Restrained_phy_HD+CBAO+Pantheon+Planck_15_nested_multi')
p = samples.getParams()

In [6]:
roots = ['Restrained_phy_CBAO+HD+Pantheon+Planck_15_nested_multi']

g= plots.getSinglePlotter(chain_dir = dir_name, width_inch=12, 
                          analysis_settings={'smooth_scale_1D': 0.5,'smooth_scale_2D': 0.6, 'ignore_rows': 0.1})

g.settings.axis_tick_x_rotation= 75
g.settings.axes_fontsize    = 10
g.settings.lab_fontsize     = 15
g.settings.alpha_filled_add = 0.9

samples = g.sampleAnalyser.samplesForRoot('Restrained_phy_CBAO+HD+Pantheon+Planck_15_nested_multi')
p = samples.getParams()

wvar   = 1.83E-11
p.weff = (-1+wvar)*np.ones(len(p.h)) 

#samples.updateBaseStatistics()
Orad   = (2.469E-5*(1+ 0*0.2271*3.046)*p.h**(-2))
Osig  = (1-p.Om)*(1+p.weff)*0.5
Odv   = (1-p.Om)*(1-p.weff)*0.5
Ode   = 1-p.Om-Orad
Oeff  = Odv + Osig
par = np.sqrt((1+ p.weff)*0.5*p.Om/Oeff)
#wx    = -2*par
#wy    = + par
gamma = -3.*par

ratio  = p.Om/Orad
zeq    = -1 + ratio
keq  = ((1-2*p.Om)*(1+zeq)**(1+3*p.weff)+2*p.Om*(1+zeq))**(0.5)*(p.h*1.0E5/299792458.)
wtmp = 1+ p.weff
x = (1 + 3.0E8)
Osbbn= 0.5*(Ode*wtmp*x**(3*wtmp)/(p.Om*x**3+Orad*x**4+Ode*x**(3*wtmp)))

def osigma(x, Om, weff, h):
    Orad   = (2.469E-5*(1+0*0.2271*3.046)*h**(-2))
    Ode  = 1-Om-Orad
    wtmp = 1+weff
    return 0.5*(Ode*wtmp*x**(3*wtmp)/(Om*x**3+Orad*x**4+Ode*x**(3*wtmp)))

def integral(x, Om, weff, h):
    # x = z + 1
    T0     = 2.7255
    return 3*T0*(np.sqrt(osigma(x, Om, weff, h))/x)*1.0E6

it = []
for i,j, k in zip(p.Om, p.weff, p.h):
    it.append(integrate.quad(integral, 1, 1101, args=(i, j, k))[0])

samples.addDerived( gamma , name='gamma', label='\gamma')
samples.addDerived( wx ,  name='wx',      label='1+w_x')
samples.addDerived( wy ,  name='wy',      label='1+w_y')
samples.addDerived( Ode , name='Ode',     label='\Omega{DE,0}')
samples.addDerived( Osig ,name='Osig',    label='\Omega_{\sigma^2,0}')
samples.addDerived( Odv , name='Odv',     label='\Omega_{dv,0}')
samples.addDerived( it ,  name='it',      label='\Delta T_{\\sigma^2}~ [\mu K]')
samples.addDerived( Oeff, name='Oeff',    label='\Omega{eff,0}')
samples.addDerived( keq , name='keq'  , label='k_{eq}')
samples.addDerived( Osbbn, name='Osbbn'  , label='\Omega_{BBN}')

samples.updateBaseStatistics()
#g.plots_1d(roots, ['weff'] , lims=[-1, -0.82, 0., 1.1])
#g.plots_1d(roots, ['gamma']) #, lims = [0, 1., 0, 1.1])
#g.plots_1d(roots, ['wx'])  
#g.plots_1d(roots, ['wy'], lims = [ -1, -0.68, 0, 1.1])
#g.plots_1d(roots, ['Osig'], lims = [ 0, 0.06, 0, 1.1])
#g.plots_1d(roots, ['Odv'])
#g.plots_1d(roots, ['it']) #, lims = [ 0, 2.3, 0, 1.1])
#g.plots_1d(roots, ['it', 'Osig', 'Odv'], nx=3)
#g.add_legend(['$w_{eff}$ = -1 + %1.1E'%wvar], legend_loc='upper right')
#plt.title('$w_{eff}$ = -1 + %1.1e'%wvar, fontsize=20)
#plt.title('$w_{\\rm eff}$ +1 = $1.83\\times10^{-11}$', fontsize=20)
#g.triangle_plot(roots, ['Odv', 'weff','h'])

#g.settings.axes_fontsize = 2
#g.settings.lab_fontsize = 10
#g.settings.colorbar_axes_fontsize=10
#g.settings.colorbar_label_rotation= 0
#g.settings.alpha_filled_add =0.9
#g.settings.scatter_size = 9
#g.settings.lw_contour=3
#g.settings.colormap_scatter= 'hsv'
#g.triangle_plot(roots, ['h','Osig', 'keq', 'it'])

g.triangle_plot(roots, 
                ['h','Osig', 'keq', 'it'], filled=True, legend_labels=['dv-wCDM']) 
#                contour_args=[{'lw':2, 'color':'blue', 'ls':'-'}, 
#                              {'lw':5, 'color':'red', 'ls':'-'}, 
#                              {'lw':2, 'color':'blue', 'ls':'-'}], filled=False,
#                alphas=[0.5, 0.6, 0.9], legend_labels=['EMLG', 'LCDM'], 
#                line_args=[{'lw':2, 'color':'blue'}, {'lw':2, 'color':'red', 'ls':'-'}, 
#                {'lw':2, 'color':'blue'}], contour_colors=['red','blue','red'])
g.add_legend(['dv-wCDM'], legend_loc='best', fontsize=12)

g.export('dv-wCDM.pdf')

/home/isidro/Downloads/chains/Restrained_phy_CBAO+HD+Pantheon+Planck_15_nested_multi_1.txt
Removed 0.1 as burn in


NameError: name 'wx' is not defined

In [7]:
print (samples.likeStats)

AttributeError: 'ParamInfo' object has no attribute 'ND_limit_bot'

In [8]:
for i,j,k in zip(samples.paramNames.list(), samples.means, samples.sddev):
    print (i,'\t \t',j,k)

TypeError: 'NoneType' object is not iterable

In [9]:
roots = ['Anisotropic_phy_CBAO+HD+Pantheon+Planck_15_nested_multi']

g= plots.getSinglePlotter(chain_dir = dir_name, width_inch=12, 
                          analysis_settings={'smooth_scale_1D': 0.4,'smooth_scale_2D': 0.5, 'ignore_rows': 0.1})

g.settings.axis_tick_x_rotation= 75
g.settings.axes_fontsize    = 10
g.settings.lab_fontsize     = 15
g.settings.alpha_filled_add = 0.9

samples = g.sampleAnalyser.samplesForRoot('Anisotropic_phy_CBAO+HD+Pantheon+Planck_15_nested_multi')
p = samples.getParams()
 

#samples.updateBaseStatistics()
Orad   = (2.469E-5*(1+0.2271*3.046)*p.h**(-2))
Ode  = 1-p.Om-Orad

ratio  = p.Om/Orad
zeq    = -1 + ratio
keq    = ((1-2*p.Om)*(1+zeq)**(-2.)+2*p.Om*(1+zeq))**(0.5)*(p.h*1.0E5/299792458.)

zbbn   = 3.0E8
Osbbn  = 3.63E-21*(1+ zbbn)**6*np.ones(len(p.Om))/(p.Om*(1+zbbn)**3+Orad*(1+zbbn)**4+ 4.0E-21*(1+ zbbn)**6)

def osigma(x, Om, h):
    Or   =  (2.469E-5*(1+0.2271*3.046)*h**(-2)) 
    Os   = 3.63E-21
    return Os*x**6/(Om*x**3+Or*x**4+ Os*x**6 + (1-Om-Or-Os))
    

def integral(x, Om, h):
    # x = z + 1
    T0     = 2.7255
    return 3*T0*(np.sqrt(osigma(x, Om, h))/x)*1.0E6


it = []
for i, j in zip(p.Om, p.h):
    tmp = integrate.quad(integral, 1, 1101, args=(i, j))[0]
    it.append(tmp)

samples.addDerived( Ode , name='Ode',     label='\Omega{DE,0}')
samples.addDerived( it ,  name='it',      label='\Delta T [\mu K]')
samples.addDerived( keq , name='keq'  , label='k_{eq}')
samples.addDerived( Osbbn, name='Osbbn'  , label='\Omega_{BBN}')

samples.updateBaseStatistics()
#g.plots_1d(roots, ['weff'] , lims=[-1, -0.82, 0., 1.1])
#g.plots_1d(roots, ['gamma']) #, lims = [0, 1., 0, 1.1])
#g.plots_1d(roots, ['wx'])  
#g.plots_1d(roots, ['wy'], lims = [ -1, -0.68, 0, 1.1])
#g.plots_1d(roots, ['Osig'], lims = [ 0, 0.06, 0, 1.1])
#g.plots_1d(roots, ['Odv'])
#g.plots_1d(roots, ['it']) #, lims = [ 0, 2.3, 0, 1.1])
#g.plots_1d(roots, ['it', 'Osig', 'Odv'], nx=3)
#g.add_legend(['$w_{eff}$ = -1 + %1.1E'%wvar], legend_loc='upper right')
#g.triangle_plot(roots, ['Odv', 'weff','h'])
#g.export('T_wCDM_wvar.pdf')

g.triangle_plot(roots, 
                ['h', 'keq', 'it'], filled=True, legend_labels=['dv-wCDM'], contour_colors=['red','blue','red']) 
#                contour_args=[{'lw':2, 'color':'blue', 'ls':'-'}, 
#                              {'lw':5, 'color':'red', 'ls':'-'}, 
#                              {'lw':2, 'color':'blue', 'ls':'-'}], filled=False,
#                alphas=[0.5, 0.6, 0.9], legend_labels=['EMLG', 'LCDM'], 
#                line_args=[{'lw':2, 'color':'blue'}, {'lw':2, 'color':'red', 'ls':'-'}, 
#                {'lw':2, 'color':'blue'}], contour_colors=['red','blue','red'])
g.add_legend(['An-$\Lambda$CDM'], legend_loc='best', fontsize=12)
g.export('An-LCDM.pdf')

/home/isidro/Downloads/chains/Anisotropic_phy_CBAO+HD+Pantheon+Planck_15_nested_multi_1.txt
Removed 0.1 as burn in


RuntimeError: Failed to process string with tex because latex could not be found

In [10]:
print (samples.likeStats)

Best fit sample -log(Like) = 526.107081
mean(-Ln(like)) = 527.647818
-Ln(mean like)  = 527.169138

parameter           bestfit        lower1         upper1         lower2         upper2
Om                  3.0041720E-01  3.1218405E-01  3.1218405E-01  3.2035932E-01  3.2035932E-01   \Omega_m
Obh2                2.2468570E-02  2.2722060E-02  2.2722060E-02  2.2848200E-02  2.2848200E-02   \Omega_{b}h^2
h                   6.8283034E-01  6.9051983E-01  6.9051983E-01  6.9498114E-01  6.9498114E-01   h
BAODR12_like       -1.6764749E+00 -1.6477752E+00 -1.6477752E+00 -1.6403270E+00 -1.6403270E+00   BAODR12
DR14LyaAuto_like   -6.5299543E-01 -5.6101135E-01 -5.6101135E-01 -5.1027182E-01 -5.1027182E-01   DR14LyaAuto
DR14LyaCross_like  -1.8946817E+00 -1.7594692E+00 -1.7594692E+00 -1.6989106E+00 -1.6989106E+00   DR14LyaCross
SixdFGS_like       -2.0532708E-04 -1.7489556E-10 -1.7489556E-10 -1.7489556E-10 -1.7489556E-10   SixdFGS
MGS_like           -1.0194980E+00 -5.6876979E-01 -5.6876979E-01 -3.2331697E-

In [11]:
for i,j,k in zip(samples.paramNames.list(), samples.means, samples.sddev):
    print (i,'\t \t',j,k)

Om 	 	 0.3012226561173623 0.006541609697819557
Obh2 	 	 0.02245366753767768 0.00014425971412791535
h 	 	 0.6822674758291803 0.0050105136351694295
BAODR12_like 	 	 -1.8493287309312167 0.2652943186331746
DR14LyaAuto_like 	 	 -0.6644840966095173 0.06709969475754979
DR14LyaCross_like 	 	 -1.901279125087994 0.08116575931049924
SixdFGS_like 	 	 -0.009493528202938778 0.015093487136112884
MGS_like 	 	 -1.0090045976300357 0.27408517097497453
eBOSS_like 	 	 -0.9672500126141148 0.5941641405212885
HD_like 	 	 -7.369042153441679 0.020473912667892898
Pantheon_like 	 	 -512.5371155534109 0.07044562705730992
SPlanck_15_like 	 	 -1.345809720161405 1.0908135714310603
Ode 	 	 0.6986875973762839 0.006542918999079511
it 	 	 20.02439427074176 0.20513323316888424
keq 	 	 0.010231741836283557 7.569838325519587e-05
Osbbn 	 	 0.7264138578614362 0.002134469454697565


In [12]:
dir_name= '/Users/josevazquezgonzalez/Desktop/Restrained/chains/Vary'

In [121]:
roots = ['LCDM_phy_HD+CBAO+Pantheon+Planck_15_nested_multi']

g= plots.getSinglePlotter(chain_dir = dir_name, width_inch=12, 
                          analysis_settings={'smooth_scale_1D': 0.4,'smooth_scale_2D': 0.5, 'ignore_rows': 0.1})


samples = g.sampleAnalyser.samplesForRoot('LCDM_phy_HD+CBAO+Pantheon+Planck_15_nested_multi')

p = samples.getParams()


Orad   = (2.469E-5*(1+0.2271*3.046)*p.h**(-2))
Ode  = 1-p.Om-Orad
ratio  = p.Om/Orad
zeq    = -1 + ratio

keq    = ((1-2*p.Om)*(1+zeq)**(-2.)+2*p.Om*(1+zeq))**(0.5)*(p.h*1.0E5/299792458.)

samples.addDerived( Ode , name='Ode',     label='\Omega{DE,0}')
samples.addDerived( keq , name='keq',     label='kde')

samples.updateBaseStatistics()

In [122]:
print (samples.likeStats)

Best fit sample -log(Like) = 526.108922
mean(-Ln(like)) = 527.648789
-Ln(mean like)  = 527.183208

parameter           bestfit        lower1         upper1         lower2         upper2
Om                  2.9994922E-01  2.8968698E-01  3.1225982E-01  2.8526568E-01  3.1825061E-01   \Omega_m
Obh2                2.2466670E-02  2.2228280E-02  2.2724690E-02  2.2092760E-02  2.2848560E-02   \Omega_{b}h^2
h                   6.8335518E-01  6.7407609E-01  6.9122737E-01  6.6969661E-01  6.9490150E-01   h
HD_like            -7.3651179E+00 -7.4140979E+00 -7.3370658E+00 -7.4485870E+00 -7.3297548E+00   HD
BAODR12_like       -1.6831227E+00 -2.3576571E+00 -1.6491053E+00 -2.9287140E+00 -1.6451814E+00   BAODR12
DR14LyaAuto_like   -6.4991286E-01 -7.9032723E-01 -5.6077070E-01 -8.6302293E-01 -5.1745746E-01   DR14LyaAuto
DR14LyaCross_like  -1.8901329E+00 -2.0399636E+00 -1.7600266E+00 -2.1097955E+00 -1.7069017E+00   DR14LyaCross
SixdFGS_like       -8.0387677E-05 -3.5420899E-02 -4.7202412E-09 -7.2699712E-02 -4

In [123]:
for i,j,k in zip(samples.paramNames.list(), samples.means, samples.sddev):
    print (i,'\t \t',j,k)

Om 	 	 0.30082325922877745 0.006486320021286634
Obh2 	 	 0.022453542031822885 0.0001493707907516196
h 	 	 0.6827046329060263 0.00502486916594694
HD_like 	 	 -7.369287274790597 0.02025759746864148
BAODR12_like 	 	 -1.8469362557014584 0.25954106791224535
DR14LyaAuto_like 	 	 -0.6645722262977313 0.06639306750164542
DR14LyaCross_like 	 	 -1.9013867099882549 0.08101193212490772
SixdFGS_like 	 	 -0.00940701754683838 0.014260230464123017
MGS_like 	 	 -1.0069236028861284 0.27401852162024587
eBOSS_like 	 	 -0.9822555859083051 0.5983983312284723
Pantheon_like 	 	 -512.5366608306385 0.06674519116246415
SPlanck_15_like 	 	 -1.3363488371328156 1.0817562091541477
Ode 	 	 0.699087109115396 0.00648762993599
keq 	 	 0.010231287930222096 7.380159618739838e-05
